In [1]:
"""
Working with the eBird API.

This notebook show you how to create a database and load observations
from the eBird API v2.0.

You will need to sign up for a key to access the eBird API.
https://ebird.org/api/keygen

You will need to copy the example file .env.example to .env
and add your API key.
"""
import os

from dotenv import load_dotenv
from sqlalchemy import create_engine

from main import settings
from main.db import models, loaders

# Load the environment variables.
load_dotenv(settings.DOTENV_FILE)
api_key = os.getenv("EBIRD_API_KEY")

In [ ]:
# Create a database and load observations using the eBird API.
# The assumption is that the database.
engine = create_engine(settings.API_DATABASE_URL)
models.Base.metadata.create_all(engine)
# The eBird API returns only the species code, e.g. "horlar1", so
# we need to populate the species table with the complete taxonomy.
loader = loaders.APILoader(api_key, settings.API_DATABASE_URL)
loader.load_taxonomy()

In [2]:
# The regions for which to load observations for. This can be any country,
# subnational1, or subnational2 code.
regions = os.getenv("EBIRD_API_REGIONS")
# The number of days to fetch checklists for.
back = int(os.getenv("EBIRD_API_PAST_DAYS"))

# Load the observations from the eBird API.
loader = loaders.APILoader(api_key, settings.API_DATABASE_URL)
loader.load(regions, back)

Fetching visits in PT-11
Number of checklists to fetch: 200
Fetching checklist S203105907 (2024-11-20)
Fetching checklist S203105931 (2024-11-20)
Fetching checklist S203106097 (2024-11-20)
Fetching checklist S203101583 (2024-11-20)
Fetching checklist S203104636 (2024-11-20)
Fetching checklist S203091615 (2024-11-20)
Fetching checklist S203083793 (2024-11-20)
Fetching checklist S203086580 (2024-11-20)
Fetching checklist S203078495 (2024-11-20)
Fetching checklist S203075721 (2024-11-20)
Fetching checklist S203071725 (2024-11-20)
Fetching checklist S203069258 (2024-11-20)
Fetching checklist S203067873 (2024-11-20)
Fetching checklist S203066616 (2024-11-20)
Fetching checklist S203065074 (2024-11-20)
Fetching checklist S203067108 (2024-11-20)
Fetching visits in PT-15
Number of checklists to fetch: 200
Fetching checklist S203104629 (2024-11-20)
Fetching checklist S203103109 (2024-11-20)
Fetching checklist S203108433 (2024-11-20)
Fetching checklist S203091913 (2024-11-20)
Fetching checklist S

In [3]:
# If you are running Linux or MacOS you can schedule the loading of
# checklists from the eBird API using cron. Here we get cron to run
# the load script at 9am, every day. Note: your crontab will be updated
# every time you execute this cell.

from crontab import CronTab
from pathlib import Path

python = Path(settings.ROOT_DIR, ".venv", "bin", "python")
script = Path(settings.ROOT_DIR, "scripts", "load.py")

with CronTab(user=True) as cron:
    job = cron.new(command=f"%s %s" % (python, script))
    job.hour.on(9)